In [20]:
import random
random.seed(123)

In [28]:
import numpy as np
import os
from datetime import datetime
import pandas as pd
from tqdm import tqdm

import datasets
import evaluate

import torch
import torch.nn as nn

from transformers import AutoTokenizer, BertConfig, BertModel, BertPreTrainedModel

from sklearn.metrics import f1_score, accuracy_score

In [1]:
MAX_LENGTH = 512

In [10]:
PRODUCT_LABELS = [
    'Vehicle loan or lease', 'Student loan', 'Consumer Loan'
]

ISSUE_LABELS = [
    'Managing the loan or lease', 'Problems at the end of the loan or lease', 'Struggling to pay your loan', 'Getting a loan or lease', 'Dealing with your lender or servicer', 'Incorrect information on your report', 'Problems when you are unable to pay', 'Taking out the loan or lease'
]

PRODUCT_LABELS_TO_IDX = {
    i : idx
    for idx, i in enumerate(PRODUCT_LABELS)
}

IDX_TO_PRODUCT_LABELS = {
    i: j
    for j, i in PRODUCT_LABELS_TO_IDX.items()
}

ISSUE_LABELS_TO_IDX = {
    i : idx
    for idx, i in enumerate(ISSUE_LABELS)
}

IDX_TO_ISSUE_LABELS = {
    i: j
    for j, i in ISSUE_LABELS_TO_IDX.items()
}


MAX_SEQUENCE_LENGTH = 512
PRETRAINED_MODEL_NAME = "distilbert-base-uncased"
NUM_TRAIN_EPOCHS = 10
FINETUNED_MODEL_PATH = './results/experiment_2024-06-19_20-51-41/checkpoint-17000'

In [15]:
class MultiTaskSentencePrediction(BertPreTrainedModel):
    def __init__(self, config, num_product_labels, num_issue_labels):
        super().__init__(config)
        self.num_product_labels = num_product_labels
        self.num_issue_labels = num_issue_labels

        self.bert = BertModel(config)

        self.product_classifier = nn.Linear(config.hidden_size, num_product_labels)
        self.issue_classifier = nn.Linear(config.hidden_size, num_issue_labels)

        classifier_dropout = config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        
        self.dropout = nn.Dropout(classifier_dropout)
        self.init_weights()

    def forward(
            self, input_ids, attention_mask=None, token_type_ids=None, 
            product_label=None, issue_label=None 
    ):
        outputs = self.bert(
            input_ids, attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        pooled_outputs = self.dropout(outputs[1])
        
        logits_product = self.product_classifier(pooled_outputs)
        logits_issue = self.issue_classifier(pooled_outputs)

        loss = None
        if product_label != None and issue_label != None:
            loss_fct1 = nn.CrossEntropyLoss()
            loss_fct2 = nn.CrossEntropyLoss()

            loss = loss_fct1(
                logits_product.view(-1, self.num_product_labels),
                product_label.view(-1)
            ) + loss_fct2(
                logits_issue.view(-1, self.num_issue_labels),
                issue_label.view(-1)
            )
        
        return (loss, logits_product, logits_issue) if loss is not None else (logits_product, logits_issue)

In [12]:
# initializing Config and Tokenizer
BERT_CONFIG = BertConfig.from_pretrained(PRETRAINED_MODEL_NAME)
TOKENIZER = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


In [18]:
# loading model 
model = MultiTaskSentencePrediction.from_pretrained(
    FINETUNED_MODEL_PATH, config=BERT_CONFIG,
    num_product_labels=len(PRODUCT_LABELS), num_issue_labels=len(ISSUE_LABELS)
)
print('Model loaded successfully...')

Model loaded successfully...


In [39]:
def get_predictions_on_dataframe(df, model):
    predictions_product = []
    predictions_issue = []

    probability_product = []
    probability_issue = []

    for text in tqdm(df.consumer_complaint, total=df.shape[0]):
        if len(text) > 0:
            inputs_to_model = TOKENIZER(text, padding=True, truncation=True)
            inputs_to_model['input_ids'] = torch.tensor([inputs_to_model['input_ids'][:MAX_SEQUENCE_LENGTH]])
            # inputs_to_model['token_type_ids'] = torch.tensor([inputs_to_model['token_type_ids'][:MAX_SEQUENCE_LENGTH]])
            inputs_to_model['attention_mask'] = torch.tensor([inputs_to_model['attention_mask'][:MAX_SEQUENCE_LENGTH]])

            with torch.no_grad():
                model_outputs = model(**inputs_to_model)
            
            overall_prob, overall_pred = torch.max(torch.softmax(model_outputs[0], dim=1), dim=1)
            second_prob, second_pred = torch.max(torch.softmax(model_outputs[1], dim=1), dim=1)
            predictions_product.append(
                IDX_TO_PRODUCT_LABELS[overall_pred.item()]
            )
            predictions_issue.append(
                IDX_TO_ISSUE_LABELS[second_pred.item()]
            )

            probability_product.append(
                round(overall_prob[0].item(), 3)
            )
            probability_issue.append(
                round(second_prob[0].item(), 3)
            )
        else:
            predictions_product.append(None)
            predictions_issue.append(None)

            probability_product.append(None)
            probability_issue.append(None)


    df['product_predictions'] = predictions_product
    df['product_pred_probability'] = probability_product
    df['issue_predictions'] = predictions_issue
    df['issue_pred_probability'] = probability_issue
    return df

In [24]:
test_df = pd.read_csv('data/test.csv')
print(test_df.shape)
test_df.head()

In [25]:
df.head()

,product,issue,consumer_complaint,overall_sentiment_predictions,overall_sentiment_pred_probability,secondary_sentiment_predictions,secondary_sentiment_pred_probability,product_predictions,product_pred_probability,issue_predictions,issue_pred_probability
0,Vehicle loan or lease,Managing the loan or lease,BMO Harris Bank engages in deceptive trade pra...,Vehicle loan or lease,0.894,Problems at the end of the loan or lease,0.605,Vehicle loan or lease,0.894,Problems at the end of the loan or lease,0.605
1,Vehicle loan or lease,Managing the loan or lease,"In XXXX XXXX, I received an equipment loan thr...",Vehicle loan or lease,0.973,Managing the loan or lease,0.877,Vehicle loan or lease,0.973,Managing the loan or lease,0.877
2,Student loan,Dealing with your lender or servicer,I am enrolled in the Public Service Loan Forgi...,Student loan,0.999,Dealing with your lender or servicer,0.945,Student loan,0.999,Dealing with your lender or servicer,0.945
3,Vehicle loan or lease,Incorrect information on your report,This is an old account reporting since 2011. T...,Student loan,0.999,Dealing with your lender or servicer,0.799,Student loan,0.999,Dealing with your lender or servicer,0.799
4,Vehicle loan or lease,Managing the loan or lease,This company have been causing me grievances s...,Vehicle loan or lease,0.980,Struggling to pay your loan,0.418,Vehicle loan or lease,0.980,Struggling to pay your loan,0.418


In [35]:
print(
    'F1 Score for Product Prediction : {}  | Issue Prediction : {}'.format(
        f1_score(y_true=df['product'], y_pred=df.product_predictions, average="weighted"),
        f1_score(y_true=df.issue, y_pred=df.issue_predictions, average='weighted')
    )
)

F1 Score for Product Prediction : 0.8350117423827834  | Issue Prediction : 0.5478804429185374


In [37]:
print(
    'Accuracy Score for Product Prediction : {}  | Issue Prediction : {}'.format(
        accuracy_score(y_true=df['product'], y_pred=df.product_predictions),
        accuracy_score(y_true=df.issue, y_pred=df.issue_predictions)
    )
)

Accuracy Score for Product Prediction : 0.8415  | Issue Prediction : 0.575
